In [ ]:
!open .

In [ ]:
import pandas as pd
import requests
import bs4
import math
import time
import NeoQuant as nq
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
pd.set_option('display.max_colwidth', -1)
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 500)

## get all company list

In [ ]:
companies = nq.get_all_company_list()
companies.index = 'A' + companies.index

## load basic dataframe from excel

In [ ]:
basic_df = pd.read_excel(r'data/basic_data.xlsx')
basic_df.index = basic_df[basic_df.columns[0]]
basic_df.index.name = ''
basic_df.drop(basic_df.columns[0], axis = 1, inplace = True)

## load fr dataframe from excel

In [ ]:
fr_df = pd.read_excel(r'data/fr_data.xlsx')
fr_df.index = fr_df[fr_df.columns[0]]
fr_df.index.name = ''
fr_df.drop(fr_df.columns[0], axis = 1, inplace = True)
big_col = list(fr_df.columns)
small_col = list(fr_df.iloc[0])

new_big_col = []
for num, col in enumerate(big_col):
    if 'Unnamed' in col:
        new_big_col.append(new_big_col[num-1])
    else:
        new_big_col.append(big_col[num])

fr_df.columns = [new_big_col, small_col]
fr_df = fr_df.loc[ fr_df.index.dropna() ]

## get base profit ratio

In [ ]:
base_profit_ratio = nq.get_base_profit_ratio()
base_profit_ratio

## if not all company

In [ ]:
# company_name_list = ['에스씨엠생명과학', '박셀바이오', '셀레믹스']
company_name_list = ['박셀바이오']

In [ ]:
company_code_list = nq.get_company_code_list(company_name_list, companies)
company_code_list

In [ ]:
fr_df = fr_df.loc[company_code_list]

## calculate adequate price

In [ ]:
fr_df = fr_df.replace('완전잠식', -1)
fr_df = fr_df.replace('N/A', -1)
fr_df = fr_df.replace('N/A(IFRS)', -1)



result_df = basic_df


total = len(fr_df)

for num, code in enumerate(fr_df.index):
    company_name = nq.get_company_name(code, companies)
    print(str(num + 1) + ' / ' + str(total) + " : " + company_name + ' (' + str(code) + ')')

    company_df = fr_df.loc[[code]]
    
    # get valid year list
    valid_year_list = []
    for col in reversed(company_df.columns.levels[0]):
        roe = pd.to_numeric(company_df.loc[code][col]['ROE'])
        net_profit = pd.to_numeric(company_df.loc[code][col]['지배주주지분']) * 100000000
        if not pd.isna(roe) and not pd.isna(net_profit):
            valid_year_list.append(col)
    
    if len(valid_year_list) == 0:
        print('>>> no valid year')
        continue

    # get roa list    
    rim_roa_list = []
    for year in valid_year_list:
        rim_roa = pd.to_numeric(company_df.loc[code][year]['RIM_ROA'])
        rim_roa_list.append(str(rim_roa))
        
    roe_list = []
    for year in valid_year_list:
        roe = pd.to_numeric(company_df.loc[code][year]['ROE'])
        roe_list.append(str(roe))
    
    recent_year = valid_year_list[0]
    roe = pd.to_numeric(company_df.loc[code][recent_year]['ROE'])
    roa = pd.to_numeric(company_df.loc[code][recent_year]['ROA'])
    net_profit = pd.to_numeric(company_df.loc[code][recent_year]['지배주주지분']) * 100000000
    per = pd.to_numeric(company_df.loc[code][recent_year]['PER'])
    pbr = pd.to_numeric(company_df.loc[code][recent_year]['PBR'])
    debt_rate = pd.to_numeric(company_df.loc[code][recent_year]['부채비율'])
    keep_rate = pd.to_numeric(company_df.loc[code][recent_year]['유보율'])
    interest_rate = pd.to_numeric(company_df.loc[code][recent_year]['배당수익률'])

    result_df.at[code, '기준일'] = recent_year
    result_df.at[code, 'valid_year_list'] = ','.join(valid_year_list)
    result_df.at[code, 'roe'] = roe
    result_df.at[code, 'roa'] = roa
    result_df.at[code, 'rim_roa_list'] = ','.join(rim_roa_list)
    result_df.at[code, 'roe_list'] = ','.join(roe_list)
    result_df.at[code, 'per'] = per
    result_df.at[code, 'pbr'] = pbr
    result_df.at[code, '부채비율'] = debt_rate
    result_df.at[code, '유보율'] = keep_rate
    result_df.at[code, '배당수익률'] = interest_rate

    result_df.at[code, 'base profit'] = base_profit_ratio
    
    stock_count = result_df.loc[code]['주식수']

#             print('stock_count : ' + str(stock_count))
    high = nq.get_more_adequate_price(net_profit, roe, base_profit_ratio, stock_count, 1)
    result_df.at[code, 'high'] = high
    middle = nq.get_more_adequate_price(net_profit, roe, base_profit_ratio, stock_count, 0.9)
    result_df.at[code, 'middle'] = middle
    low = nq.get_more_adequate_price(net_profit, roe, base_profit_ratio, stock_count, 0.8)
    result_df.at[code, 'low'] = low
    very_low = nq.get_more_adequate_price(net_profit, roe, base_profit_ratio, stock_count, 0.5)
    result_df.at[code, 'very_low'] = very_low


try:
    result_df['dis_rate'] = result_df['price'] / result_df['low']
    result_df = result_df.sort_values(by='dis_rate', ascending=True)
except KeyError:
    print('>>>>> KeyError')


## 3년 동안 ROA 10 이상, 배당 수익률 1 이상  필터링

In [ ]:
filtered_list = []
for code in result_df.index:
#     print(code)
    company_df = result_df.loc[code]
    if type(company_df['rim_roa_list']) != str:
#         print(code + 'no roa list')
        continue
    roa_list = company_df['rim_roa_list'].split(',')
    if len(roa_list) < 3:
        continue
        
    bingo = True
    for roa in roa_list:
        if float(roa) < 10:
            bingo = False
            break
    if bingo:
        filtered_list.append(code)
    
# print(filtered_list)

filtered_df = result_df.loc[filtered_list]
filtered_df = filtered_df.loc[filtered_df['배당수익률'] >= 1]

## 3년 동안 ROE 가 상향 중

In [ ]:
filtered_list = []
for code in filtered_df.index:
#     print(code)
    company_df = filtered_df.loc[code]
    if type(company_df['roe_list']) != str:
        print(code + 'no roe list')
        continue
    roe_list = company_df['roe_list'].split(',')
    if len(roe_list) < 3:
        continue
        
    new_roe_list = []
    for roe in reversed(roe_list):
        new_roe_list.append(float(roe))

    if nq.generally_increasing(new_roe_list, 2):
        filtered_list.append(code)
    
# print(filtered_list)

filtered_df = filtered_df.loc[filtered_list]

In [ ]:
filtered_df

## make report

In [ ]:
final_df = filtered_df
final_df = final_df.loc[~final_df.index.str.startswith('A9')]
final_df['dis_rate'] = final_df['price'] / final_df['low']
final_df = final_df.sort_values(by='dis_rate', ascending=True)

In [ ]:
final_df['rank'] = final_df['dis_rate'].rank()

In [ ]:
final_df = final_df.loc[final_df['roe'] > 0]

In [ ]:
final_df

## save all data

In [ ]:
final_df.to_excel(r'data/s_rim_roa_filtered.xlsx')

## filtering

In [ ]:
final_df = final_df.loc[final_df['dis_rate'] > 0.1]
final_df = final_df.loc[final_df['dis_rate'] < 1]
final_df = final_df.loc[final_df['roe'] > final_df['expectation']]

In [ ]:
final_df = final_df.loc[final_df['roe'] > 10]
final_df = final_df.loc[final_df['roe'] < 100]
final_df = final_df.loc[final_df['시가총액'] > 1000 * 100000000]

In [ ]:
final_df = final_df.loc[final_df['배당수익률'] > 1]

In [ ]:
final_df = final_df.loc[~final_df['name'].str.contains('홀딩스')]

In [ ]:
final_df = final_df.loc[final_df['roe'] > 10]

In [ ]:
len(final_df)

## save port

In [ ]:
final_df.to_excel(r'data/s_rim_roa_filtered.xlsx')

## load port

In [ ]:
# port_df = pd.read_excel(r'data/port_data.xlsx')
port_df = pd.read_excel(r'data/s_rim_roa_filtered.xlsx')
port_df.index = port_df[port_df.columns[0]]
port_df.index.name = ''
port_df.drop(port_df.columns[0], axis = 1, inplace = True)